In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, get_train, get_test
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'TR'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,_,_ = pickle.load(f)

n_subs = np.max(params[:,0])

In [14]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven24'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = np.max(params[:,0])
data_load = True

for sub in range(1,n_subs+1):
    sess.sub = sub
    tf.keras.backend.clear_session()

    train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
    test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)

    n_dof = np.max(params[train_ind,4])
    
    # Train NNs
    mlp = MLP(n_class=n_dof)
    mlp_beta = MLPbeta(n_class=n_dof)
    cnn = CNN(n_class=n_dof)

    optimizer = tf.keras.optimizers.Adam()

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
    
    if np.sum(train_ind) and np.sum(test_ind):

        if data_load:
            with open('noisedata_all_emgscalelim/' + str(sess.sub_type) + str(sub) + '_grp_2_fullallmix4_5.p','rb') as f:
                scaler, x_train_cnn, _, _, _, y_train, _, x_train_lda, y_train_lda, x_train_aug, y_train_aug = pickle.load(f)
        
            x_train_cnn = x_train_cnn.astype('float32')
            x_train_mlp = x_train_cnn.reshape(x_train_cnn.shape[0],-1)
            
            trainmlp = tf.data.Dataset.from_tensor_slices((x_train_mlp, y_train)).shuffle(x_train_mlp.shape[0],reshuffle_each_iteration=True).batch(128)
            traincnn = tf.data.Dataset.from_tensor_slices((x_train_cnn, y_train)).shuffle(x_train_cnn.shape[0],reshuffle_each_iteration=True).batch(128)
        
            sess.scaler = scaler
        else:   
            trainmlp, validmlp, traincnn, validcnn, y_train, y_valid, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, y_train_aug = prd.prep_train_data(sess,raw,params)

        models = [mlp, mlp_beta, cnn]

        for model in models:

            if isinstance(model,CNN):
                ds = traincnn
            else:
                ds = trainmlp
            
            train_mod = get_train()

            for epoch in range(sess.epochs):
                # Reset the metrics at the start of the next epoch
                train_loss.reset_states()
                train_accuracy.reset_states()

                for x, y in ds:
                    train_mod(x, y, model, optimizer, train_loss, train_accuracy)

                if epoch == 0 or epoch == sess.epochs-1:
                    print(
                        f'Epoch {epoch + 1}, '
                        f'Loss: {train_loss.result():.2f}, '
                        f'Accuracy: {train_accuracy.result() * 100:.2f} '
                    )
            
            del train_mod
        
        # Train aligned
        y_train_aligned = np.argmax(y_train, axis=1)[...,np.newaxis]

        mlp_enc = mlp.get_layer(name='enc')
        w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aligned)

        mlpbeta_enc = mlp_beta.get_layer(name='enc')
        w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aligned)
        
        cnn_enc = cnn.get_layer(name='enc')
        temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
        temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
        w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aligned)

        # Train LDA
        w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
        w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

        # Load test data
        if data_load:
            with open('testdata_all_emgscalelim_noisescalelim/TR' + str(sub) + '_grp_2_' + str(sess.test) + '_1.p','rb') as f:
                x_test_cnn, _, x_test_lda, y_test = pickle.load(f) 
            x_temp = np.transpose(x_test_lda.reshape((x_test_lda.shape[0],4,-1)),(0,2,1))[...,np.newaxis]
            x_test_cnn = sess.scaler.transform(x_temp.reshape(x_temp.shape[0]*x_temp.shape[1],-1)).reshape(x_temp.shape)
            x_test_cnn = x_test_cnn.astype('float32')

            # Reshape for nonconvolutional SAE
            x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)
        else:
            with open('real_noise/all_real_noise.p', 'rb') as f:
                real_noise_temp, _ = pickle.load(f)

            x_test_cnn, x_test_mlp, x_test_lda, y_test, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)
       
        # Test
        mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
        mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
        cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
        y_test_aligned = np.argmax(y_test, axis=1)[...,np.newaxis]

        clean_lda, noisy_lda = eval_lda(w, c, x_test_lda, y_test_aligned, clean_size)

        clean_aug, noisy_aug = eval_lda(w_aug, c_aug, x_test_lda, y_test_aligned, clean_size)

        clean_mlp, noisy_mlp = eval_nn(x_test_mlp, y_test,mlp,clean_size)
        clean_mlplda, noisy_mlplda = eval_lda(w_mlp, c_mlp, mlp_test_aligned, y_test_aligned, clean_size)

        clean_mlpb, noisy_mlpb = eval_nn(x_test_mlp, y_test,mlp_beta,clean_size)
        clean_mlpblda, noisy_mlpblda = eval_lda(w_mlpbeta, c_mlpbeta, mlpbeta_test_aligned, y_test_aligned, clean_size)

        clean_cnn, noisy_cnn = eval_nn(x_test_cnn, y_test,cnn,clean_size)
        clean_cnnlda, noisy_cnnlda = eval_lda(w_cnn, c_cnn, cnn_test_aligned, y_test_aligned, clean_size)
        print( 
            f'LDA ---- '
            f'Clean: {clean_lda * 100:.2f}, '
            f'Noisy: {noisy_lda * 100:.2f}'
            f'\nAUG ---- '
            f'Clean: {clean_aug * 100:.2f}, '
            f'Noisy: {noisy_aug * 100:.2f}'
            f'\nMLP ---- '
            f'Clean: {clean_mlp * 100:.2f}, '
            f'Noisy: {noisy_mlp * 100:.2f}, '
            f'LDA Clean: {clean_mlplda * 100:.2f}, '
            f'LDA Noisy: {noisy_mlplda * 100:.2f}'
            f'\nMLPB ---- '
            f'Clean: {clean_mlpb * 100:.2f}, '
            f'Noisy: {noisy_mlpb * 100:.2f}, '
            f'LDA Clean: {clean_mlpblda * 100:.2f}, '
            f'LDA Noisy: {noisy_mlpblda * 100:.2f}'
            f'\nCNN ---- '
            f'Clean: {clean_cnn * 100:.2f}, '
            f'Noisy: {noisy_cnn * 100:.2f}, '
            f'LDA Clean: {clean_cnnlda * 100:.2f}, '
            f'LDA Noisy: {noisy_cnnlda * 100:.2f}'
        )
    else:
        print('no training or testing data')

Epoch 1, Loss: 1.89, Accuracy: 25.11 
Epoch 30, Loss: 0.86, Accuracy: 65.94 
Epoch 1, Loss: 1.28, Accuracy: 49.91 
Epoch 30, Loss: 0.41, Accuracy: 84.32 
Epoch 1, Loss: 1.20, Accuracy: 53.35 
Epoch 30, Loss: 0.34, Accuracy: 87.00 
LDA ---- Clean: 62.45, Noisy: 43.86
AUG ---- Clean: 42.60, Noisy: 44.67
MLP ---- Clean: 58.90, Noisy: 58.25, LDA Clean: 58.80, LDA Noisy: 57.00
MLPB ---- Clean: 62.80, Noisy: 58.39, LDA Clean: 60.90, LDA Noisy: 57.69
CNN ---- Clean: 59.95, Noisy: 58.58, LDA Clean: 61.50, LDA Noisy: 60.14
Epoch 1, Loss: 1.83, Accuracy: 25.30 
Epoch 30, Loss: 0.62, Accuracy: 73.93 
Epoch 1, Loss: 1.22, Accuracy: 52.60 
Epoch 30, Loss: 0.26, Accuracy: 89.44 
Epoch 1, Loss: 1.07, Accuracy: 61.13 
Epoch 30, Loss: 0.23, Accuracy: 91.07 
LDA ---- Clean: 85.20, Noisy: 66.53
AUG ---- Clean: 65.75, Noisy: 59.69
MLP ---- Clean: 73.25, Noisy: 70.06, LDA Clean: 74.35, LDA Noisy: 72.31
MLPB ---- Clean: 80.60, Noisy: 77.39, LDA Clean: 82.80, LDA Noisy: 79.81
CNN ---- Clean: 80.30, Noisy: 76

In [ ]:
Loading training data: traindata_all/TR1_traindata_2.p
Loading training data: traindata_all/TR1_traindata_4.p
LDA ---- Clean: 41.82, Noisy: 21.61
AUG ---- Clean: 14.29, Noisy: 17.89
MLP ---- Clean: 51.82, Noisy: 39.29, LDA Clean: 52.86, LDA Noisy: 41.29
MLPB ---- Clean: 58.00, Noisy: 46.14, LDA Clean: 56.64, LDA Noisy: 46.46
CNN ---- Clean: 58.32, Noisy: 47.61, LDA Clean: 60.00, LDA Noisy: 48.79
Loading training data: traindata_all/TR2_traindata_2.p
Loading training data: traindata_all/TR2_traindata_4.p
LDA ---- Clean: 69.43, Noisy: 36.14
AUG ---- Clean: 15.79, Noisy: 20.00
MLP ---- Clean: 67.39, Noisy: 50.82, LDA Clean: 69.00, LDA Noisy: 52.57
MLPB ---- Clean: 74.25, Noisy: 59.29, LDA Clean: 72.64, LDA Noisy: 61.57
CNN ---- Clean: 73.86, Noisy: 61.29, LDA Clean: 73.96, LDA Noisy: 60.96
Loading training data: traindata_all/TR3_traindata_2.p
Loading training data: traindata_all/TR3_traindata_4.p
LDA ---- Clean: 67.35, Noisy: 26.25
AUG ---- Clean: 21.15, Noisy: 27.20
MLP ---- Clean: 64.95, Noisy: 48.35, LDA Clean: 66.00, LDA Noisy: 49.15
MLPB ---- Clean: 65.40, Noisy: 54.35, LDA Clean: 65.65, LDA Noisy: 56.80
CNN ---- Clean: 62.75, Noisy: 49.35, LDA Clean: 64.95, LDA Noisy: 52.25
Loading training data: traindata_all/TR4_traindata_2.p
Loading training data: traindata_all/TR4_traindata_4.p
LDA ---- Clean: 58.20, Noisy: 28.40
AUG ---- Clean: 20.00, Noisy: 25.20
MLP ---- Clean: 69.55, Noisy: 55.05, LDA Clean: 71.75, LDA Noisy: 56.45
MLPB ---- Clean: 81.20, Noisy: 66.05, LDA Clean: 82.20, LDA Noisy: 67.05
CNN ---- Clean: 77.90, Noisy: 64.00, LDA Clean: 76.00, LDA Noisy: 63.50